In [1]:
import pandas as pd 

# Setting up the data

In [2]:
# Loading the energy experiments 

df1 = pd.read_csv("opower_experiment_1.csv")
df2 = pd.read_csv("opower_experiment_2.csv")
df3 = pd.read_csv("opower_experiment_3.csv")
df4 = pd.read_csv("opower_experiment_4.csv")
df5 = pd.read_csv("opower_experiment_5.csv")
df6 = pd.read_csv("opower_experiment_6.csv")
df7 = pd.read_csv("opower_experiment_7.csv")

df1["cluster_id"] = 1  
df2["cluster_id"] = 2
df3["cluster_id"] = 3  
df4["cluster_id"] = 4  
df5["cluster_id"] = 5  
df6["cluster_id"] = 6  
df7["cluster_id"] = 7  


In [3]:
df_array = [df1, df2, df3] 

In [4]:
energy_dfs_silo = []
true_ates = []

for i in range(len(df_array)):
    # Leave-one-out setup
    leave_out_df = df_array[i]
    stay_in_dfs = pd.concat([df_array[j] for j in range(len(df_array)) if j != i])

    # Label S = 1 for experimental sample, S = 0 for target population
    stay_in_dfs["S"] = 1
    leave_out_df["S"] = 0

    # Rename variables
    stay_in_dfs = stay_in_dfs.rename(columns={"treatment": "T", "daily_kwh": "Y"})
    leave_out_df = leave_out_df.rename(columns={"treatment": "T", "daily_kwh": "Y"})

    # Concatenate trial and target
    energy_dfs = pd.concat([stay_in_dfs, leave_out_df])

    # Save combined dataset
    energy_dfs_silo.append(energy_dfs)

    # === Compute true ATE in left-out (target) sample ===
    target = leave_out_df
    treated = target[target["T"] == 1]["Y"]
    control = target[target["T"] == 0]["Y"]
    ate = treated.mean() - control.mean()
    true_ates.append(ate)


## Setting up the Estimators

In [5]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

# === Estimators ===
from methods.ipsw import estimate_ate_ipsw

from methods.aipsw import estimate_ate_aipsw
from methods.outcomeM import estimate_ate_outcome

from methods.stratification import StratifiedGeneralizabilityEstimator 

from methods.calibration import estimate_ate_calibration
from methods.acalibration import estimate_ate_calibration_augmented

# === Utilities ===
from methods.bootstrap_utils import run_ate_with_bootstrap, run_clustered_bootstrap

# === Models ===
from sklearn.linear_model import LogisticRegression, LinearRegression
from xgboost import XGBClassifier, XGBRegressor


In [6]:
def ipsw_logit_estimator(Y, T, S, X, **kwargs):
    """
    IPSW estimator with logistic regression for estimating P(S=1 | X).

    Parameters
    ----------
    Y : array-like
        Outcome variable.
    T : array-like
        Treatment assignment.
    S : array-like
        Trial sample indicator (1=trial, 0=target).
    X : array-like or pd.DataFrame
        Covariates used to predict S.
    kwargs :
        Additional arguments to pass to estimate_ate_ipsw().

    Returns
    -------
    result : object
        Object with `.ate` attribute (and possibly others).
    """
    logit_model = LogisticRegression(solver="lbfgs", max_iter=1000)
    logit_model.fit(X, S)
    ps_hat = logit_model.predict_proba(X)[:, 1]

    result = estimate_ate_ipsw(
        y=Y,
        a=T,
        s=S,
        ps=ps_hat,
        weight_type="inverse_odds",
        stabilized=True,
        clip=(0.01, 50),
        **kwargs,
    )
    return result

def ipsw_xgb_estimator(Y, T, S, X, **kwargs):
    """
    IPSW estimator with XGBoost for estimating P(S=1 | X).

    Parameters
    ----------
    Y : array-like
        Outcome variable.
    T : array-like
        Treatment assignment.
    S : array-like
        Trial sample indicator (1=trial, 0=target).
    X : array-like or pd.DataFrame
        Covariates used to predict S.
    kwargs :
        Additional arguments to pass to estimate_ate_ipsw().

    Returns
    -------
    result : object
        Object with `.ate` attribute (and possibly others).
    """
    xgb_model = XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        random_state=42,
    )
    xgb_model.fit(X, S)
    ps_hat = xgb_model.predict_proba(X)[:, 1]

    result = estimate_ate_ipsw(
        y=Y,
        a=T,
        s=S,
        ps=ps_hat,
        weight_type="inverse_odds",
        stabilized=True,
        clip=(0.01, 50),
        **kwargs,
    )
    return result

def stratified_logit_estimator(Y, T, S, X, n_strata=5, **kwargs):
    """
    Stratified ATE estimator using logistic regression for sampling score model.

    Parameters
    ----------
    Y : array-like
        Outcome variable.
    T : array-like
        Treatment indicator.
    S : array-like
        Sample indicator (1 = trial, 0 = target).
    X : array-like (DataFrame or ndarray)
        Covariates.
    n_strata : int
        Number of strata (default: 5).
    **kwargs : optional
        Additional keyword arguments passed to StratifiedGeneralizabilityEstimator.

    Returns
    -------
    result : object
        An object with attributes:
        - .ate
        - .stratum_ates
        - .stratum_weights
    """
    model = LogisticRegression(solver="lbfgs", max_iter=1000)

    est = StratifiedGeneralizabilityEstimator(
        n_strata=n_strata,
        target="nontrial",
        sampling_model=model,
        **kwargs,
    )

    est.fit(X=X, S=S)
    result = est.estimate_ate(Y=Y, T=T, S=S)
    return result

def aipsw_logit_xgb_estimator(Y, T, S, X, **kwargs):
    """
    AIPSW with:
      - sampling model: Logistic regression (S ~ X)
      - outcome model:  XGBoost regressor (Y ~ X, within trial)
    """
    # Define models
    logit_sampler = LogisticRegression(solver="lbfgs", max_iter=1000)
    xgb_outcome = XGBRegressor(
        n_estimators=500,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
    )

    # Call your AIPSW convenience function
    res = estimate_ate_aipsw(
        y=Y,
        a=T,
        s=S,
        X=X,
        sampling_model=logit_sampler,
        outcome_model=xgb_outcome,
        target="nontrial",
        **kwargs,
    )
    return res

def aipsw_logit_linear_estimator(Y, T, S, X, **kwargs):
    logit_sampler = LogisticRegression(solver="lbfgs", max_iter=1000)
    linear_outcome = LinearRegression()

    result = estimate_ate_aipsw(
        y=Y,
        a=T,
        s=S,
        X=X,
        sampling_model=logit_sampler,
        outcome_model=linear_outcome,
        target="nontrial",
        **kwargs,
    )
    return result

def aipsw_logit_xgb_estimator(Y, T, S, X, **kwargs):
    logit_sampler = LogisticRegression(solver="lbfgs", max_iter=1000)
    xgb_outcome = XGBRegressor(
        n_estimators=500,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
    )

    result = estimate_ate_aipsw(
        y=Y,
        a=T,
        s=S,
        X=X,
        sampling_model=logit_sampler,
        outcome_model=xgb_outcome,
        target="nontrial",
        **kwargs,
    )
    return result

def aipsw_xgb_linear_estimator(Y, T, S, X, **kwargs):
    xgb_sampler = XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        random_state=42,
    )
    linear_outcome = LinearRegression()

    result = estimate_ate_aipsw(
        y=Y,
        a=T,
        s=S,
        X=X,
        sampling_model=xgb_sampler,
        outcome_model=linear_outcome,
        target="nontrial",
        **kwargs,
    )
    return result

def aipsw_xgb_xgb_estimator(Y, T, S, X, **kwargs):
    xgb_sampler = XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        random_state=42,
    )
    xgb_outcome = XGBRegressor(
        n_estimators=500,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
    )

    result = estimate_ate_aipsw(
        y=Y,
        a=T,
        s=S,
        X=X,
        sampling_model=xgb_sampler,
        outcome_model=xgb_outcome,
        target="nontrial",
        **kwargs,
    )
    return result

def outcome_linear_estimator(Y, T, S, X, **kwargs):
    """
    Estimates ATE using an outcome model approach with linear regression.

    Only uses trial units (S=1) to fit models for μ₁(X) and μ₀(X),
    then extrapolates to target (S=0).

    Returns a result object with .ate
    """
    model = LinearRegression()

    result = estimate_ate_outcome(
        Y=Y,
        T=T,
        S=S,
        X=X,
        outcome_model=model,
        target="nontrial",
        **kwargs,
    )
    return result

def outcome_xgb_estimator(Y, T, S, X, **kwargs):
    """
    Estimates ATE using an outcome model approach with XGBoost.

    Only uses trial units (S=1) to fit models for μ₁(X) and μ₀(X),
    then extrapolates to target (S=0).

    Returns a result object with .ate
    """
    model = XGBRegressor(
        n_estimators=500,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
    )

    result = estimate_ate_outcome(
        Y=Y,
        T=T,
        S=S,
        X=X,
        outcome_model=model,
        target="nontrial",
        **kwargs,
    )
    return result

def calib_estimator(Y, T, S, X, **kwargs):
    """
    Calibration weighting estimator

    Returns:
        result with `.ate`
    """

    result = estimate_ate_calibration(
        Y=Y,
        T=T,
        S=S,
        X=X,
        target="nontrial",
        **kwargs,
    )
    return result

def aug_calib_xgb_estimator(Y, T, S, X, **kwargs):
    """
    Augmented calibration estimator using:
    - XGBoost regressor for outcome model

    Returns:
        result with `.ate`
    """

    xgb_outcome = XGBRegressor(
        n_estimators=500,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
    )

    result = estimate_ate_calibration_augmented(
        Y=Y,
        T=T,
        S=S,
        X=X,
        outcome_model=xgb_outcome,
        target="nontrial",
        **kwargs,
    )
    return result

def aug_calib_linear_estimator(Y, T, S, X, **kwargs):
    """
    Augmented calibration estimator using:
    - XGBoost regressor for outcome model

    Returns:
        result with `.ate`
    """

    linear_model = LinearRegression()

    result = estimate_ate_calibration_augmented(
        Y=Y,
        T=T,
        S=S,
        X=X,
        outcome_model=linear_model,
        target="nontrial",
        **kwargs,
    )
    return result

def get_latex_table(estimator_list, estimator_names, X_list, df, n_boot=400, alpha=0.05):
    """
    Run a list of estimators and return a LaTeX table of ATE, SE, and CI.

    Parameters
    ----------
    estimator_list : list of callables
        List of estimator functions (each should accept Y, T, S, X).
    estimator_names : list of str
        List of estimator names (same order as estimator_list).
    X_list : list of str
        List of column names in df to use as covariates.
    df : pd.DataFrame
        DataFrame with columns Y, T, S, and X covariates.
    n_boot : int
        Number of bootstrap repetitions.
    alpha : float
        Significance level for confidence intervals (default 0.05).

    Returns
    -------
    latex_str : str
        LaTeX-formatted table string.
    """
    results = []

    for name, estimator in zip(estimator_names, estimator_list):
        try:
            ate, se, lo, hi = run_ate_with_bootstrap(
                estimator=estimator,
                Y=df["Y"].values,
                T=df["T"].values,
                S=df["S"].values,
                X=df[X_list],
                n_boot=n_boot,
                alpha=alpha,
            )
            results.append({
                "Estimator": name,
                "ATE": round(ate, 3),
                "SE": round(se, 3),
                "95% CI": f"[{lo:.3f}, {hi:.3f}]"
            })
        except Exception as e:
            results.append({
                "Estimator": name,
                "ATE": "ERROR",
                "SE": "ERROR",
                "95% CI": str(e)
            })

    df_results = pd.DataFrame(results)

    latex_str = df_results.to_latex(index=False, caption="Estimated ATEs with Bootstrapped CIs", label="tab:ate_bootstrap")
    return latex_str

def get_latex_table_clustered(estimator_list, estimator_names, X_list, df, cluster_col, n_boot=400, alpha=0.05):
    """
    Run a list of estimators and return a LaTeX table of ATE, SE, and CI
    using clustered bootstrap by experiment/group.

    Parameters
    ----------
    estimator_list : list of callables
        Each estimator function must accept (Y, T, S, X) and return an ATE.
    estimator_names : list of str
        Labels for each estimator.
    X_list : list of str
        Covariate columns to use from df.
    df : pd.DataFrame
        DataFrame with columns: Y, T, S, X covariates, and cluster_col.
    cluster_col : str
        Name of column identifying cluster (e.g., experiment or group).
    n_boot : int
        Number of bootstrap samples.
    alpha : float
        Confidence level (e.g., 0.05 → 95% CI).

    Returns
    -------
    latex_str : str
        LaTeX-formatted table string.
    """
    results = []
    cluster_ids = df[cluster_col].values
    Y = df["Y"].values
    T = df["T"].values
    S = df["S"].values
    X = df[X_list]

    for name, estimator in zip(estimator_names, estimator_list):
        try:
            ate, se, lo, hi = run_clustered_bootstrap(
                estimator_fn=estimator,
                Y=Y,
                T=T,
                S=S,
                X=X,
                cluster_ids=cluster_ids,
                n_boot=n_boot,
                alpha=alpha,
            )
            results.append({
                "Estimator": name,
                "ATE": round(ate, 3),
                "SE": round(se, 3),
                "95% CI": f"[{lo:.3f}, {hi:.3f}]"
            })
        except Exception as e:
            results.append({
                "Estimator": name,
                "ATE": "ERROR",
                "SE": "ERROR",
                "95% CI": str(e)
            })

    df_results = pd.DataFrame(results)

    latex_str = df_results.to_latex(index=False, caption="Estimated ATEs (Clustered Bootstrap)", label="tab:ate_clustered")
    return latex_str


In [7]:
X_vars = ['temp','alpha_i','is_high_usage', 'lambda_t']

df = energy_dfs_silo[0]

latex_output = get_latex_table(
    estimator_list=[
        ipsw_logit_estimator,
        ipsw_xgb_estimator,
        stratified_logit_estimator, 
        aipsw_logit_linear_estimator,
        aipsw_logit_xgb_estimator,
        aipsw_xgb_xgb_estimator,
        aipsw_xgb_linear_estimator,
        outcome_linear_estimator,
        outcome_xgb_estimator,
        calib_estimator,
        aug_calib_linear_estimator,
        aug_calib_xgb_estimator
    ],
    
    estimator_names=[
        "IPSW (Logit)",
        "IPSW (XGBoost)",
        "Stratified (Logit)",
        "AIPSW (Logit x Linear)",
        "AIPSW (Logit x XGBRegressor)",
        "AIPSW (XGBClassifer x XGBRegressor)",
        "AIPSW (XGBClassifer x Linear)",
        "G-Formulation (Linear)",
        "G-Formulation (XGBRegressor)", 
        "Calibration Weighing",
        "Aug. Calibration Weighing (Linear)",
        "Aug. Calibration Weighing (XGBRegressor)",
    ],
    X_list=X_vars,
    df=df,
    n_boot=50,
    alpha=0.05,
)

print(latex_output)


\begin{table}
\caption{Estimated ATEs with Bootstrapped CIs}
\label{tab:ate_bootstrap}
\begin{tabular}{lrrl}
\toprule
Estimator & ATE & SE & 95% CI \\
\midrule
IPSW (Logit) & -0.996000 & 0.012000 & [-1.021, -0.976] \\
IPSW (XGBoost) & -1.111000 & 0.020000 & [-1.124, -1.054] \\
Stratified (Logit) & -1.007000 & 0.012000 & [-1.031, -0.987] \\
AIPSW (Logit x Linear) & -0.964000 & 0.006000 & [-0.973, -0.953] \\
AIPSW (Logit x XGBRegressor) & -0.964000 & 0.007000 & [-0.977, -0.951] \\
AIPSW (XGBClassifer x XGBRegressor) & -0.964000 & 0.007000 & [-0.977, -0.952] \\
AIPSW (XGBClassifer x Linear) & -0.963000 & 0.006000 & [-0.972, -0.951] \\
G-Formulation (Linear) & -0.964000 & 0.006000 & [-0.973, -0.953] \\
G-Formulation (XGBRegressor) & -0.964000 & 0.007000 & [-0.977, -0.951] \\
Calibration Weighing & -0.996000 & 0.012000 & [-1.021, -0.976] \\
Aug. Calibration Weighing (Linear) & -0.964000 & 0.006000 & [-0.973, -0.953] \\
Aug. Calibration Weighing (XGBRegressor) & -0.963000 & 0.007000 & [-0.97

In [8]:
latex_output = get_latex_table_clustered(
    estimator_list=[
        ipsw_logit_estimator,
        ipsw_xgb_estimator,
        stratified_logit_estimator, 
        aipsw_logit_linear_estimator,
        aipsw_logit_xgb_estimator,
        aipsw_xgb_xgb_estimator,
        aipsw_xgb_linear_estimator,
        outcome_linear_estimator,
        outcome_xgb_estimator,
        calib_estimator,
        aug_calib_linear_estimator,
        aug_calib_xgb_estimator
    ],
    
    estimator_names=[
        "IPSW (Logit)",
        "IPSW (XGBoost)",
        "Stratified (Logit)",
        "AIPSW (Logit x Linear)",
        "AIPSW (Logit x XGBRegressor)",
        "AIPSW (XGBClassifer x XGBRegressor)",
        "AIPSW (XGBClassifer x Linear)",
        "G-Formulation (Linear)",
        "G-Formulation (XGBRegressor)", 
        "Calibration Weighing",
        "Aug. Calibration Weighing (Linear)",
        "Aug. Calibration Weighing (XGBRegressor)",
    ],
    X_list=X_vars,
    cluster_col= 'cluster_id',
    df=df,
    n_boot=100,
    alpha=0.05,
)

print(latex_output)

\begin{table}
\caption{Estimated ATEs (Clustered Bootstrap)}
\label{tab:ate_clustered}
\begin{tabular}{lrrl}
\toprule
Estimator & ATE & SE & 95% CI \\
\midrule
IPSW (Logit) & -0.996000 & 0.307000 & [-1.326, -0.591] \\
IPSW (XGBoost) & -1.111000 & 0.302000 & [-1.374, -0.615] \\
Stratified (Logit) & -1.007000 & 0.232000 & [-1.261, -0.703] \\
AIPSW (Logit x Linear) & -0.964000 & 0.231000 & [-1.213, -0.660] \\
AIPSW (Logit x XGBRegressor) & -0.964000 & 0.229000 & [-1.231, -0.683] \\
AIPSW (XGBClassifer x XGBRegressor) & -0.964000 & 0.229000 & [-1.231, -0.682] \\
AIPSW (XGBClassifer x Linear) & -0.963000 & 0.231000 & [-1.214, -0.660] \\
G-Formulation (Linear) & -0.964000 & 0.231000 & [-1.213, -0.660] \\
G-Formulation (XGBRegressor) & -0.964000 & 0.229000 & [-1.231, -0.683] \\
Calibration Weighing & -0.996000 & 0.307000 & [-1.326, -0.591] \\
Aug. Calibration Weighing (Linear) & -0.964000 & 0.231000 & [-1.213, -0.660] \\
Aug. Calibration Weighing (XGBRegressor) & -0.963000 & 0.228000 & [-1.23

In [9]:
print(true_ates[0])

-0.7326964420450324
